### Loading Libraries

In [1]:
import os

In [2]:
import random
import string

In [3]:
import pandas as pd

In [42]:
import requests

In [47]:
from bs4 import BeautifulSoup as bs

In [22]:
from nltk.tag import StanfordPOSTagger

### Stanford POS tagger

In [24]:
## Setting the path to java.exe for nltk
java_path = "F:/java/java/bin/java.exe"
    
os.environ['JAVAHOME'] = java_path

In [33]:
## creating a StanfordPOStagger object
model_filename = "E:\Stanford Taggers\stanford-postagger-full-2015-04-20\stanford-postagger-full-2015-04-20\models"
path_to_jar = "E:\Stanford Taggers\stanford-postagger-full-2015-04-20\stanford-postagger-full-2015-04-20"

st = StanfordPOSTagger(model_filename+"\english-bidirectional-distsim.tagger", path_to_jar+"\stanford-postagger.jar")

## Functions

In [18]:
# function to return a random plot form data frame with Title and Genre
def sel_plot(df):
    # choosing a random value 
    num = random.choice([100, 150, 200, 250])
    val = random.randrange(1,num)

    print(f"Selected value : {val}")
    
    ## re-framing the data
    plots = {}
    for index,row in df.iterrows():
        plots[row["Title"]] = {"Genre":row["Genre"], "Plot": row["Plot"]}
        if index == val: break
            
    # choosing a random title
    titles = list(plots.keys())
    sel_movie = random.choice(titles)
    
    return plots[sel_movie]

In [186]:
## function to tokenize the plot into Parts of Speech
# func to get abb of pos
def get_abb(tag_obj):
    tag_dct = {}
    
    if type(tag_obj) == str: tags = st.tag(tag_obj.split())
    elif type(tag_obj) == list: tags = st.tag(tag_obj)
    for word,tag in tags:
        tag_dct[word] = tag,abb[tag] 
        
    return tag_dct 

# func to get tokens
def pos_tokenizer(plot_obj):
    plot = plot_obj["Plot"]
    get_abb(plot)
      
    return plot_obj["Genre"],tag_dct

In [104]:
# pos_tokenizer(sel_plot(req_data))

### Fetching Data

In [6]:
## Fetching the movie data
movies_data = pd.read_csv("wiki_movie_plots_deduped.csv", encoding="utf-8")

req_data = movies_data[["Title", "Genre", "Plot"]]
req_data = req_data[req_data["Genre"] != "unknown"]

In [13]:
## Cleaning the data
# cleaning the plot encoding
def clean_str(st):
    for ch in st :
        if ch in string.punctuation:
            st = st.replace(ch,"")
    st = st.encode("ascii", "ignore").decode("utf-8")
    return st

req_data["Plot"] = req_data["Plot"].apply(lambda x:clean_str(x))

### POS tags Abrrevations

In [56]:
url = "https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html"
res = requests.get(url)
soup = bs(res.text)

In [85]:
table = soup.select("table tr")

abb = {}
for row in table[1:]:
    cols = row.find_all("td")
    col_lst = [col.get_text() for col in cols][1:]
    abb[col_lst[0].strip()] = col_lst[1].strip()

In [97]:
# abb

### nltk Stopwords

In [157]:
from nltk.corpus import stopwords

In [197]:
eng_wrds = stopwords.words("english")

In [198]:
eng_abb = get_abb(tmp_wrds[:100])

In [266]:
eng_dct = {}

for key,value in eng_abb.items():
    if value[1] not in eng_dct:
        eng_dct[value[1]] = list()
    eng_dct[value[1]].append(key)      

## StoryGenerator

In [142]:
res = pos_tokenizer(sel_plot(req_data))

Selected value : 10


In [143]:
genre = res[0]
pos_tup = res[1]

In [145]:
words = list(pos_tup.keys())

In [146]:
num = random.randrange(1, 20)
choosen = []
for i in range(num):
    choosen.append(random.choice(words))

In [154]:
# choosen
new_words = []
for title,tup in pos_tup.items():
    if title in choosen:
        new_word = input(f"Enter a {tup[1]}:")
        new_words.append(new_word)
    else:
        new_words.append(title)

Enter a Determiner:a
Enter a Verb, 3rd person singular present:live
Enter a Noun, singular or mass:Ben
Enter a Verb, non-3rd person singular present:eat
Enter a Determiner:the
Enter a Adjective:brave
Enter a Verb, non-3rd person singular present:jump
Enter a Verb, base form:love
Enter a Adjective:angry
Enter a Noun, singular or mass:sally
Enter a Verb, 3rd person singular present:drank
Enter a Possessive pronoun:his
Enter a Noun, singular or mass:pen


In [155]:
" ".join(new_words)

'a film live about a Ben who eat to the suburbs hoping for brave life Things jump love angry and sally gets violent drank throwing crockery leading his pen'